# 感知器模型 Perceptron

感知器是1957年由Frank Rosenblatt 提出的一种单层人工神经网络模型，被视为神经网络与机器学习领域的奠基性工作之一。

其本质是一种线性二分类器，通过调整权重对输入数据进行类别划分，为后续多层神经网络（如多层感知机MLP）的发展奠定了基础。

下面是经典4×4感知器模型区分字母T,J的例子

```python
T = [
    [1,1,1,0,
     0,1,0,0,
     0,1,0,0,
     0,1,0,0,],
    
    [0,1,1,1,
     0,0,1,0,
     0,0,1,0,
     0,0,1,0,],
]
J = [
    [0,0,1,0,
     0,0,1,0,
     1,0,1,0,
     1,1,1,0,],
    
    [0,0,0,1,
     0,0,0,1,
     0,1,0,1,
     0,1,1,1,],
]
```

In [1]:
T = [[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]]
J = [[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1]]

# 权重层
w = [0.0 for _ in range(16)]
# 截距项
b = 0.0


# 输出
def perceptron_output(inputs, w, b):
    z = 0
    for i in range(16):
        z += inputs[i] * w[i] + b
    return z


print("训练前")
print("T:", [perceptron_output(t, w, b) for t in T])
print("J:", [perceptron_output(j, w, b) for j in J])

# 训练参数
eval_iters = 200
learning_rate = 0.1

# 首先 定义 T 输出值 -1 ,J 输出值 1

T_OUT = -1
J_OUT = 1

# 执行梯度下降
for _ in range(eval_iters):
    # 预测值
    dw_list = []
    db_list = []
    pred = perceptron_output(T[0], w, b)
    dw_list.append([(pred - T_OUT) * i for i in T[0]])
    db_list.append(pred - T_OUT)
    pred = perceptron_output(T[1], w, b)
    dw_list.append([(pred - T_OUT) * i for i in T[1]])
    db_list.append(pred - T_OUT)
    pred = perceptron_output(J[0], w, b)
    dw_list.append([(pred - J_OUT) * i for i in J[0]])
    db_list.append(pred - J_OUT)
    pred = perceptron_output(J[1], w, b)
    dw_list.append([(pred - J_OUT) * i for i in J[1]])
    db_list.append(pred - J_OUT)
    dw = []
    for i in range(16):
        dw.append(sum(dw_list[j][i] for j in range(4)) / 4)
    db = sum(db_list) / 4
    w = [w[i] - learning_rate * dw[i] for i in range(16)]
    b -= learning_rate * db

# 训练结果
print("训练后")
print("T:", [perceptron_output(t, w, b) for t in T])
print("J:", [perceptron_output(j, w, b) for j in J])

训练前
T: [0.0, 0.0]
J: [0.0, 0.0]
训练后
T: [-1.0000003207718833, -0.9999967553958412]
J: [0.9999972307565017, 0.9999997979499405]


可以看到我们之前的定义 T 输出值 < 0 ,J 输出值 > 0 

经过 200 次梯度下降训练之后 输入 T 的值几乎为 -1，而 J 的值几乎为 1 

可以说训练成功

# 张量 Tensor

张量是多维数组的泛化形式，用于表示高维空间中的线性关系数据。其阶数（Rank）定义了维度层级，是深度学习框架（如TensorFlow、PyTorch）的核心数据结构。

我们发现，在实现感知器模型代码里，存在一些不足之处，例如：

- 对不同的输入和输出，单独处理，导致代码量增多
- 不同的输入输出互不依赖可并行计算，但额外实现并行会变得非常复杂
- 计算梯度的候每次更新权重总要把权重和输入显式遍历。

上述问题我们均可以使用张量运算进行简化表示。

1. 把不同的输入和输出压合并在一起
2. 输入和权重行列线性运算时可直接表示为矩阵乘法
3. 求梯度时直接进行张量点乘批量运算

上面代码直接表示为如下形式，变得十分简洁。

In [2]:
import torch

T = [[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]]
J = [[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1]]
inputs = torch.stack((torch.tensor(T, dtype=torch.float), torch.tensor(J, dtype=torch.float)))
outputs = torch.tensor([[-1, -1], [1, 1]], dtype=torch.float)
# 权重层
w = torch.zeros(16)
# 截距项
b = 0.0


# 输出
def perceptron_output(inputs, w, b):
    return inputs @ w + b

print("训练前")
print(perceptron_output(inputs, w, b))

# 训练参数
eval_iters = 200
learning_rate = 0.1

# 执行梯度下降
for _ in range(eval_iters):
    pred = perceptron_output(inputs, w, b)
    err = pred - outputs
    b -= learning_rate * err.mean()
    dw = err.unsqueeze(-1) * inputs
    w -= learning_rate * dw.mean(dim=[0, 1])

# 训练结果
print("训练后")
print(perceptron_output(inputs, w, b))

训练前
tensor([[0., 0.],
        [0., 0.]])
训练后
tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]])


# 关于 PyTorch

上面我们清晰感受到了 PyTorch 诸如之类的深度学习框架对模型实现的简化能力。

PyTorch 作为当前主流的深度学习框架之一，其动态计算图机制和自动微分系统彻底改变了传统梯度计算的实现方式。

在标准PyTorch训练中，开发者只需关注三个核心要素：

1. 张量运算：通过torch.Tensor实现数据的高效存储与并行计算
2. 自动微分：利用autograd模块自动追踪运算图的梯度信息
3. 优化器体系：通过torch.optim提供SGD/Adam等优化算法的标准化实现

下面是使用 PyTorch 实现感知器模型范例

下面的代码实际上是 PyTorch 的范式，我们在代码内每步都进行了注释

In [3]:
T = [[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]]
J = [[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1]]
import torch

inputs = torch.stack((torch.tensor(T, dtype=torch.float), torch.tensor(J, dtype=torch.float)))
outputs = torch.tensor([[-1, -1], [1, 1]], dtype=torch.float)

# 训练参数
eval_iters = 100
learning_rate = 0.1

model = torch.nn.Linear(16, 1) # 感知器模型本质上就是一个线性层

print("训练前")
print(model(inputs))

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 执行梯度下降
for _ in range(eval_iters):
    pred = model(inputs).squeeze(-1) # 线性层输出张量大小指定为 1 意味着输出为形状为 [1] 的向量而不是标量，这里做一下调整，对计算没有影响
    loss = torch.nn.functional.mse_loss(pred, outputs) # 用一种合适的方式计算梯度下降方向，这里用的标准差，但梯度下降方向之前实现的均差方向一致
    optimizer.zero_grad() # 清除算法梯度
    loss.backward() # 损失向后传播
    optimizer.step() # 更新模型（线性层）参数
    
# 训练结果

print("训练后")
print(model(inputs))

训练前
tensor([[[0.5146],
         [0.2753]],

        [[0.5183],
         [0.3044]]], grad_fn=<ViewBackward0>)
训练后
tensor([[[-1.0000],
         [-1.0000]],

        [[ 1.0000],
         [ 1.0000]]], grad_fn=<ViewBackward0>)
